## Librerías 

In [17]:
import numpy as np
import joblib ### para cargar array

########Paquetes para NN #########
import tensorflow as tf
from sklearn import metrics ### para analizar modelo
from sklearn.ensemble import RandomForestClassifier  ### para analizar modelo
import pandas as pd
from sklearn.utils import class_weight
from sklearn import tree
import a_funciones as fn

import cv2 ### para leer imagenes jpeg
### pip install opencv-python

from matplotlib import pyplot as plt #

In [18]:
# cargar bases_procesadas ##
x_train, y_train, x_test, y_test, x_val, y_val = fn.imag_array()

100%|██████████| 227/227 [00:01<00:00, 128.93it/s]


In [2]:
# cargar bases_procesadas #

x_train = joblib.load('salidas/x_train.pkl')
y_train = joblib.load('salidas/y_train.pkl')
x_test = joblib.load('salidas/x_test.pkl')
y_test = joblib.load('salidas/y_test.pkl')



FileNotFoundError: [Errno 2] No such file or directory: 'salidas/x_train.pkl'

In [19]:
x_train.max()

255

In [20]:
x_test.max()

255

In [21]:
#### Escalar ######################
x_train = x_train.astype('float32') ## para poder escalarlo
x_test = x_test.astype('float32') ## para poder escalarlo
x_train /= 255 ### escalaro para que quede entre 0 y 1
x_test /= 255

###### verificar tamaños
print(x_train.shape)
x_test.shape

(2372, 224, 224, 3)


(336, 224, 224, 3)

In [22]:
np.product(x_train[1].shape) ## cantidad de variables por imagen

150528

In [23]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([1569,  803]))

In [24]:
np.unique(y_test, return_counts=True)

(array([0, 1]), array([208, 128]))

In [25]:
##### convertir a 1 d array ############
x_train2 = x_train.reshape(2372,150528)
x_test2 = x_test.reshape(336, 150528)
print(x_train2.shape)
x_test2.shape

(2372, 150528)


(336, 150528)

### Selección de métricas 

Recall: Se selecciona el Recall como métrica de rendimiento para nuestro modelo de predicción de cáncer de mama porque queremos minimizar los falsos negativos (casos en los que el modelo predice incorrectamente que una persona no tiene cáncer cuando en realidad sí lo tiene). El Recall mide la proporción de casos positivos correctamente identificados, asegurando que detectemos la mayor cantidad posible de personas con cáncer, lo cual es esencial en aplicaciones de salud donde un diagnóstico omitido puede tener consecuencias graves.

AUC: Teniendo en cuenta que esta métrica permite obtener información sobre qué tan capaz es el modelo para identificar casos positivos vs una mala detección o calificación de negativos. Entonces, para nuestro problema, su aplicabilidad iría en determinar el porcentaje de las pacientes con cáncer de mama que el modelo logró identificar correctamenre (TP, verdaderos positivos) en comparación con aquellas que en realidad estaban sanas, pero que el modelo predice como cáncer (FP, falsos positivos).  Esta métrica es útil ya que, aunque los falsos negativos tienen un impacto alto y severo a nivel de bienestar e integridad de la persona (lo cual puede llegar a tener consecuencias graves, hasta costarle la vida); por otro lado, el hecho de tener falsos positivos, supone compromisos a nivel psicológico y costos financieros tanto para el hospital/clínica como para la paciente. Adicionalmente, en dado caso no se hagan más pruebas para descartar el cáncer de mama, recetar medicina para este tipo de patologías a personas sanas o con diagnósticos distintos puede alterar de igual forma su calidad de vida. Y supone una pérdida de credibilidad para la entidad de salud.

### 1. Modelo Random Forest

In [31]:
modelo_1=RandomForestClassifier() #bosques aleatorios
modelo_1.fit(x_train2, y_train) #entrenar modelo

train_pred=modelo_1.predict(x_train2)
print(metrics.classification_report(y_train, train_pred)) #importante el RECALL de train


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1569
           1       1.00      1.00      1.00       803

    accuracy                           1.00      2372
   macro avg       1.00      1.00      1.00      2372
weighted avg       1.00      1.00      1.00      2372

1.0


In [36]:
print(f"La métrica AUC para entrenamiento dio: {metrics.roc_auc_score(y_train, train_pred)}") #AUC de train

La métrica AUC dio: 1.0


In [40]:
test_pred=modelo_1.predict(x_test2)
print(metrics.classification_report(y_test, test_pred)) #importante el RECALL de test

              precision    recall  f1-score   support

           0       0.62      0.94      0.75       208
           1       0.43      0.08      0.13       128

    accuracy                           0.61       336
   macro avg       0.53      0.51      0.44       336
weighted avg       0.55      0.61      0.51       336



In [41]:
print(f"La métrica AUC para evaluación dio: {metrics.roc_auc_score(y_test, test_pred)}") #AUC de test

La métrica AUC dio: 0.5078125


### 2. Modelo de Redes Neuronales Estándar

In [ ]:
y_train = y_train.ravel() #convierte y_train en un arreglo unidimensional


# Definir el modelo de red neuronal con regularización Dropout
fc_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=x_train.shape[1:]),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),  # Dropout para reducir el sobreajuste; 'apaga' el 30% de neuronas
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

c:\Users\delva\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
#Configurar el optimizador
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) #busca minimizar la función de pérdida del modelo


In [12]:
# Calcular los pesos de cada clase para mejorar el aprendizaje de las clases desbalanceadas
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

#Compilar el modelo
fc_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['Recall', 'AUC'])


In [13]:
#Entrenamiento del modelo
fc_model.fit(
    x_train, y_train,
    batch_size=32,  #tamaño de lote reducido
    epochs=20,  #mas iteraciones para una mejor convergencia
    validation_data=(x_test, y_test),
    class_weight=class_weights_dict
)

# Evaluar el modelo
test_loss, test_recall, test_auc = fc_model.evaluate(x_test, y_test, verbose=2)
print("Test recall:", test_recall)
print("Test AUC:", test_auc)
print("Test loss:", test_loss)


Epoch 1/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 11s 76ms/step - AUC: 0.5110 - Recall: 0.4653 - loss: 0.8581 - val_AUC: 0.5198 - val_Recall: 0.0859 - val_loss: 0.6776
Epoch 2/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 9s 56ms/step - AUC: 0.4899 - Recall: 0.4487 - loss: 0.7191 - val_AUC: 0.5231 - val_Recall: 0.4688 - val_loss: 0.6916
Epoch 3/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - AUC: 0.4942 - Recall: 0.6712 - loss: 0.6986 - val_AUC: 0.5087 - val_Recall: 1.0000 - val_loss: 0.6949
Epoch 4/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - AUC: 0.4869 - Recall: 0.7689 - loss: 0.6979 - val_AUC: 0.5101 - val_Recall: 0.9531 - val_loss: 0.6963
Epoch 5/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - AUC: 0.5035 - Recall: 0.7750 - loss: 0.6902 - val_AUC: 0.4965 - val_Recall: 0.9922 - val_loss: 0.6940
Epoch 6/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 6s 54ms/step - AUC: 0.5123 - Recall: 0.9445 - loss: 0.6956 - val_AUC: 0.4973 - val_Recall: 1.0000 - val_loss: 0.6942
Epoch 7/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - AUC: 0.5042 - Re